<a href="https://colab.research.google.com/github/RizkiNoor16/ind-sentiment-analysyt-nlp/blob/main/Create_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RizkiNoor16/ind-sentiment-analysyt-nlp.git

Cloning into 'ind-sentiment-analysyt-nlp'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), 382.52 KiB | 2.48 MiB/s, done.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
df = pd.read_csv('/content/ind-sentiment-analysyt-nlp/Prepocess-Dataset/Dataset.csv')
df.head()

,Unnamed: 0,Tweet,Label,Panjang_tweet,Jumlah_kalimat
0,0,trs barusan dia reach out aku trs minta tolong...,Fear,122,22
1,1,sumpah ini receh bgt dan not bts related. cm j...,Fear,272,48
2,2,ragu utk bisa beli barang tu karn harganya bis...,Fear,68,12
3,3,bisa gak sih langsung tanggal 11 aja gausah ad...,Fear,92,18
4,4,iya kaan yg lain suka pada coba pd gpp tp tete...,Fear,58,13


In [4]:
from sklearn.model_selection import train_test_split
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.10

train_tweet, val_tweet, train_label, val_label = train_test_split(df['Tweet'],
                                                                  df['Label'],
                                                                  test_size = 1-train_ratio,
                                                                  random_state=42)

val_tweet, test_tweet, val_label, test_label = train_test_split(val_tweet,
                                                                val_label,
                                                                test_size =test_ratio/(test_ratio + val_ratio),
                                                                random_state=42)

print(f'Total training sampel {len(train_tweet)} tweet & {len(train_label)} label')
print(f'Total val sampel {len(val_tweet)} tweet & {len(val_label)} label')
print(f'Total test sampel {len(test_tweet)}tweet & {len(test_label)} label')

Total training sampel 5310 tweet & 5310 label
Total val sampel 1062 tweet & 1062 label
Total test sampel 708tweet & 708 label


In [5]:
train_label

2131        Joy
5679    Neutral
6210        Sad
5157    Neutral
7031        Sad
         ...   
3772       Love
5191    Neutral
5226    Neutral
5390    Neutral
860        Fear
Name: Label, Length: 5310, dtype: object

In [6]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_label.to_numpy().reshape(-1,1))
val_labels_one_hot = one_hot_encoder.transform(val_label.to_numpy().reshape(-1,1))
test_label_one_hot = one_hot_encoder.transform(test_label.to_numpy().reshape(-1,1))

train_labels_one_hot

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_label_encoded = label_encoder.fit_transform(train_label.to_numpy())
val_label_encoded = label_encoder.transform(val_label.to_numpy())
test_label_encoded = label_encoder.transform(test_label.to_numpy())

print(train_label_encoded)

[2 4 5 ... 4 4 1]


In [8]:
num_class = len(label_encoder.classes_)
class_names = label_encoder.classes_

num_class, class_names

(6, array(['Anger', 'Fear', 'Joy', 'Love', 'Neutral', 'Sad'], dtype=object))

#Model 0 : Baseline using Naive Bayes

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([('tf-idf', TfidfVectorizer()),
                    ('clf', MultinomialNB())])

model_0.fit(X= train_tweet,
            y= train_label_encoded)

Pipeline(steps=[('tf-idf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [13]:
model_0.score(X= val_tweet,
              y = val_label_encoded)

0.487758945386064

In [21]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
def results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [22]:
pred_base = model_0.predict(val_tweet)
pred_base

array([4, 4, 4, ..., 4, 4, 5])

In [24]:
hasil_baseline = results(y_true=val_label_encoded,
                                     y_pred=pred_base)
hasil_baseline

{'accuracy': 48.7758945386064,
 'precision': 0.7296387469898832,
 'recall': 0.487758945386064,
 'f1': 0.45495581638111265}

In [28]:
hasil = pd.DataFrame({"baseline": hasil_baseline})
hasil = hasil.transpose()
hasil

,accuracy,f1,precision,recall
baseline,48.775895,0.454956,0.729639,0.487759
